# Diving into French Presidential Discourses
## Valérie Pécresse (VP)

In [ ]:
!pip install minet
!pip install pdfminer.six
!pip install unidecode
!python -m spacy download fr_core_news_sm

In [1]:
import pandas as pd
from pdfminer.high_level import extract_text
from gensim.test.utils import datapath
from gensim import utils
import spacy
from collections import Counter
import unidecode
import gensim.models

In [2]:
## create a .csv of VP's Twitter activity since 27 Jan 2022
!minet tw scrape tweets "(from:vpecresse) until:2022-03-27 since:2022-01-27" > tweets_VP.csv

## convert the .csv file in a data frame using pandas
df_tw_VP = pd.read_csv("./tweets_VP.csv")

## create a list of tweets selecting the 'text' column of the data frame
list_tw_VP = df_tw_VP['text'].values.tolist()
len(list_tw_VP)

Searching for "(from:vpecresse) until:2022-03-27 since:2022-01-27"


425

In [3]:
## retrieve VP's affiliates1 Twitter activity
!minet tw scrape tweets "(from:MichelBarnier OR from:ChJacob77 OR from:ADublanche OR from:othmannasrou OR from:ECiotti OR from:xavierbertrand OR from:DebordValerie OR from:gerard_larcher) until:2022-03-27 since:2022-01-27" > tw_VP_aff_all1.csv

## convert VP's affiliates' tweets in a list
df_tw_VP_aff_all1 = pd.read_csv("tw_VP_aff_all1.csv")
list_tw_VP_aff_all1 = df_tw_VP_aff_all1['text'].values.tolist()
print(list_tw_VP_aff_all1[0])
print(len(list_tw_VP_aff_all1))

Searching for "(from:MichelBarnier OR from:ChJacob77 OR from:ADublanche OR from:othmannasrou OR from:ECiotti OR from:xavierbertrand OR from:DebordValerie OR from:gerard_larcher) until:2022-03-27 since:2022-01-27"
L'Etat ne doit pas faire de bénéfice sur le dos des Français du fait de la hausse des prix du carburant. @vpecresse propose de rendre l’intégralité de la hausse de taxes liée à l’augmentation du prix du carburant, soit 25 centimes par litre. #Pécresse2022 https://twitter.com/othmannasrou/status/1507832408212656131/video/1
1482


In [4]:
## retrieve VP's affiliates2 Twitter activity
!minet tw scrape tweets "(from:VincentJEANBRUN OR from:nadine__morano OR from:rezeg_hamida OR from:AurelienPradie OR from:CharlesConsigny OR from:GeoffroyDidier OR from:BrunoRetailleau OR from:GuilhemCarayon OR from:Herve_Morin) until:2022-03-24 since:2022-01-27" > tw_VP_aff_all2.csv

## convert VP's affiliates' tweets in a list
df_tw_VP_aff_all2 = pd.read_csv("tw_VP_aff_all2.csv")
list_tw_VP_aff_all2 = df_tw_VP_aff_all2['text'].values.tolist()
print(list_tw_VP_aff_all2[0])
print(len(list_tw_VP_aff_all2))

Searching for "(from:VincentJEANBRUN OR from:nadine__morano OR from:rezeg_hamida OR from:AurelienPradie OR from:CharlesConsigny OR from:GeoffroyDidier OR from:BrunoRetailleau OR from:GuilhemCarayon OR from:Herve_Morin) until:2022-03-24 since:2022-01-27"
Le Pécresse bashing continue contre l’évidence : @vpecresse fait une excellente émission ce soir dans @FaceaBaba « JulesTorres17: Après deux séquences difficiles face à @MarionMarechal et @Fabien_Roussel, @vpecresse s’apprête à débattre avec l’insoumis @AQuatennens.
#FaceÀBaba — https://twitter.com/JulesTorres17/status/1506754260997922825 »
1045


In [5]:
## merge the two lists of affiliates' tweets
list_tw_VP_aff_all = list_tw_VP_aff_all1 + list_tw_VP_aff_all2
print(len(list_tw_VP_aff_all))

## merge VP and his affiliates' lists
list_tw_VP_all = list_tw_VP + list_tw_VP_aff_all
print(len(list_tw_VP_all))

2527
2952


In [6]:
## i retrieve a string from the pdf of VP's manifesto using extract_text of the pdfminer package
## the cleaning process is specific for this manifesto and it depends on the output of extract_text
manif_VP = extract_text('/Users/simonemariaparazzoli/Documents/Università/Sciences Po/Diving into public digital spaces/research/manifesto_pecresse.pdf')
manif_clean_VP = manif_VP.replace('-\n','')
manif_clean_VP = manif_clean_VP.replace(' \n',' ')
manif_clean_VP = manif_clean_VP.replace('\n ','')
manif_clean_VP = manif_clean_VP.replace('\x0c',' ')
manif_clean_VP = manif_clean_VP.replace('\n\n','\n')
manif_clean_VP = manif_clean_VP.replace('\n','---')
#manif_clean_MLP = manif_clean_MLP.replace('\xa0','')
#print(repr(manif_clean_VP))

## convert the string of the manifesto into a list
list_manif_VP = manif_clean_VP.split("---")
list_manif_VP = [s for s in list_manif_VP if len(s)>30]
len(list_manif_VP)
#print(list_manif_MLP)

298

In [12]:
## merge the two lists of tweets and of the manifesto 
list_VP = list_tw_VP_all + list_tw_VP_all + list_manif_VP
len(list_VP)

## load a spacy model to retrieve stop words
nlp = spacy.load("fr_core_news_sm")
stop_words_fr = nlp.Defaults.stop_words

## clean the list of tweets and manifesto to get rid of useless words and make the list content homogeneous
list_VP_clean = []
for i in list_VP:
    doc = nlp(i)
    tokens = [unidecode.unidecode(token.text).lower() for token in doc 
              if (token.text not in stop_words_fr and
                  len(token.text)>2 and
                  token.like_url == False )]
    tokens_joined = ' '.join(tokens)
    list_VP_clean.append(tokens_joined)
    
## test the output of the cleaning process
print(list_VP[205])
print("---")
print(list_VP_clean[205])

📍Visite de la scierie Gaiffe à Champ-le-Duc et échanges sur les enjeux économiques et territoriaux de la filière bois. #Pecresse2022 https://twitter.com/vpecresse/status/1500143607365476359/photo/1
---
visite scierie gaiffe champ-le-duc echanges enjeux economiques territoriaux filiere bois pecresse2022


In [90]:
## prepare the corpus as a class
class MyCorpus_VP:

    def __iter__(self):
        for i in list_VP_clean:
            yield utils.simple_preprocess(i,min_len=3)
            
## train the word embeddings model_MLP
sentences = MyCorpus_VP()
model_VP = gensim.models.Word2Vec(sentences=sentences, min_count=10, vector_size=300, epochs=100)

In [91]:
model_VP.wv.most_similar("president")

[('addition', 0.33905965089797974),
 ('contraire', 0.31321820616722107),
 ('reelection', 0.3054465651512146),
 ('seule', 0.28812894225120544),
 ('promet', 0.2842770516872406),
 ('sortant', 0.27787160873413086),
 ('qui', 0.2763318717479706),
 ('emmanuel', 0.26000869274139404),
 ('risque', 0.2541629374027252),
 ('lettre', 0.2537105679512024)]

In [92]:
## transform the corpus list (that is made of tweets and sentences from the manifesto)
## in a list containing all the words of the corpus as elements of the list
words_VP = []

for i in list_VP_clean:
    i_split = i.split(' ') #transform the i document into a list (split at blank space)
    words_VP.extend(i_split)

## clean the list of tokens
words_VP_clean = [x for x in words_VP
                   if x not in stop_words_fr
                   if x != "\n\n"
                   if len(x)>1]

## find the 30 most common words using Counter
words_freq_VP = Counter(words_VP_clean)
common_words_VP = words_freq_VP.most_common(30)
print(common_words_VP)

[('@vpecresse', 1458), ('pecresse2022', 1422), ('france', 1165), ('macron', 909), ('francais', 593), ('faire', 476), ('emmanuel', 453), ('ans', 422), ('projet', 390), ('soir', 388), ('pays', 378), ('nouvellefrance', 322), ('pecresse', 309), ('ukraine', 308), ('zemmour', 303), ("aujourd'hui", 300), ('campagne', 289), ('faut', 287), ('veux', 286), ('president', 286), ('ete', 281), ('politique', 277), ('face', 261), ('contre', 255), ('valerie', 248), ('soutien', 241), ('region', 236), ('republique', 225), ('etat', 222), ('temps', 221)]


In [93]:
## first attempt with the most_similar function on our corpus using our model_MLP
result = model_VP.wv.most_similar(positive=['realite'], topn=30)
print(result)

[('concret', 0.3169578015804291), ('oui', 0.2994443476200104), ('camp', 0.28076252341270447), ('puisse', 0.27869153022766113), ('aller', 0.2784546911716461), ('connaitre', 0.27111056447029114), ('consideration', 0.26271265745162964), ('bouquet', 0.2551777958869934), ('qofficiel', 0.2527967393398285), ('diplomatie', 0.2523679733276367), ('macronisme', 0.25222375988960266), ('trop', 0.25202009081840515), ('zigzag', 0.24884957075119019), ('croire', 0.24453037977218628), ('hommes', 0.2443080097436905), ('bureau', 0.2429942488670349), ('entendre', 0.2419843226671219), ('facealaguerretf', 0.2413264364004135), ('confiance', 0.24130795896053314), ('islam', 0.2393571436405182), ('probleme', 0.23890991508960724), ('salariale', 0.2379254549741745), ('scene', 0.23328234255313873), ('desordre', 0.22915305197238922), ('energetique', 0.2288573980331421), ('perdre', 0.22837014496326447), ('hopital', 0.22802335023880005), ('par', 0.22625865042209625), ('poissy', 0.22434157133102417), ('retrouver', 0.22

In [94]:
print(len(list_tw_VP_all),',',len(list_VP_clean))

2952 , 6202


In [95]:
result = model_VP.wv.most_similar(positive=['france'], topn=20)
print(result)

[('pays', 0.31511548161506653), ('defends', 0.30728644132614136), ('identite', 0.30477091670036316), ('libre', 0.2829281687736511), ('puissante', 0.2607462704181671), ('fier', 0.25054290890693665), ('concorde', 0.24943602085113525), ('gaullienne', 0.2466525286436081), ('oui', 0.24615752696990967), ('culture', 0.2458539456129074), ('yeux', 0.2423001378774643), ('nation', 0.24006387591362), ('lumieres', 0.2347973883152008), ('soyons', 0.23471394181251526), ('vrai', 0.23395942151546478), ('cheffe', 0.23191852867603302), ('puissance', 0.23144982755184174), ('rassembler', 0.2309294044971466), ('leve', 0.22943666577339172), ('imposer', 0.22664976119995117)]


In [96]:
# this is the old one! with no changes on epochs parameters
result = model_VP.wv.most_similar(positive=['france'], topn=20)
print(result)

[('pays', 0.31511548161506653), ('defends', 0.30728644132614136), ('identite', 0.30477091670036316), ('libre', 0.2829281687736511), ('puissante', 0.2607462704181671), ('fier', 0.25054290890693665), ('concorde', 0.24943602085113525), ('gaullienne', 0.2466525286436081), ('oui', 0.24615752696990967), ('culture', 0.2458539456129074), ('yeux', 0.2423001378774643), ('nation', 0.24006387591362), ('lumieres', 0.2347973883152008), ('soyons', 0.23471394181251526), ('vrai', 0.23395942151546478), ('cheffe', 0.23191852867603302), ('puissance', 0.23144982755184174), ('rassembler', 0.2309294044971466), ('leve', 0.22943666577339172), ('imposer', 0.22664976119995117)]


In [97]:
result = model_VP.wv.most_similar(positive=['etat'], topn=20)
print(result)

[('explose', 0.3032076358795166), ('societes', 0.28143635392189026), ('lemondefr', 0.27409738302230835), ('impots', 0.2710159718990326), ('comment', 0.26956668496131897), ('cabinet', 0.26772457361221313), ('raisons', 0.240378737449646), ('quelqu', 0.23662935197353363), ('libertes', 0.23502913117408752), ('liberer', 0.2348335087299347), ('dette', 0.2346971482038498), ('parleurs', 0.23380449414253235), ('locales', 0.22938229143619537), ('cas', 0.22887560725212097), ('faible', 0.2286521941423416), ('ferai', 0.22815896570682526), ('structurelles', 0.22602276504039764), ('croire', 0.22531147301197052), ('paye', 0.22477570176124573), ('commercial', 0.2239103466272354)]


In [98]:
result = model_VP.wv.most_similar(positive=['souverainete'], topn=20)
print(result)

[('alimentaire', 0.4665467143058777), ('pilier', 0.3997895121574402), ('survie', 0.3980392515659332), ('enjeu', 0.3944235146045685), ('majeur', 0.39420080184936523), ('industrielle', 0.3718438744544983), ('consequences', 0.35222527384757996), ('rang', 0.3509189486503601), ('independance', 0.34814801812171936), ('entreprises', 0.3395942747592926), ('europeens', 0.33604538440704346), ('juridique', 0.3192204535007477), ('preuve', 0.3182990252971649), ('unis', 0.3105933964252472), ('strategique', 0.3012201189994812), ('tirer', 0.3001767694950104), ('energetique', 0.2998339831829071), ('etats', 0.29584261775016785), ('produire', 0.2939181327819824), ('defense', 0.28955787420272827)]


In [99]:
result = model_VP.wv.most_similar(positive=['president'], topn=20)
print(result)

[('addition', 0.33905965089797974), ('contraire', 0.31321820616722107), ('reelection', 0.3054465651512146), ('seule', 0.28812894225120544), ('promet', 0.2842770516872406), ('sortant', 0.27787160873413086), ('qui', 0.2763318717479706), ('emmanuel', 0.26000869274139404), ('risque', 0.2541629374027252), ('lettre', 0.2537105679512024), ('mandat', 0.2522598206996918), ('fausses', 0.2490234076976776), ('spectacle', 0.24675194919109344), ('attaquer', 0.24427586793899536), ('macron', 0.24104396998882294), ('monde', 0.23864053189754486), ('fermer', 0.2373092770576477), ('reelu', 0.2353917509317398), ('meeting', 0.2346964031457901), ('difference', 0.23388127982616425)]


In [100]:
result = model_VP.wv.most_similar(positive=['politique'], topn=20)
print(result)

[('responsabilite', 0.28362536430358887), ('amene', 0.25962021946907043), ('autant', 0.25714606046676636), ('sinceres', 0.2500777244567871), ('islamisme', 0.22663180530071259), ('droite', 0.210000678896904), ('proces', 0.20971426367759705), ('generation', 0.20910727977752686), ('essentiel', 0.20592685043811798), ('mur', 0.20564067363739014), ('cela', 0.2033241093158722), ('rassembler', 0.20330996811389923), ('agir', 0.19951441884040833), ('simple', 0.1953183263540268), ('election', 0.19373999536037445), ('europeen', 0.19061627984046936), ('appelle', 0.18783998489379883), ('difficultes', 0.1875726729631424), ('macroniste', 0.18738868832588196), ('immobilisme', 0.18621709942817688)]


In [101]:
result = model_VP.wv.most_similar(positive=['droit'], topn=20)
print(result)

[('savent', 0.4501812160015106), ('moreno', 0.3852461576461792), ('foot', 0.3716205358505249), ('leguer', 0.3682292103767395), ('sol', 0.36423972249031067), ('voulu', 0.3507581651210785), ('ifopopinion', 0.3284815847873688), ('mode', 0.31607770919799805), ('fasse', 0.3127777874469757), ('exprimer', 0.30220237374305725), ('voile', 0.2839970290660858), ('souhaitent', 0.2804102897644043), ('regions', 0.26953157782554626), ('dur', 0.26612940430641174), ('idee', 0.26097890734672546), ('islamistes', 0.2607285678386688), ('quitter', 0.2607143521308899), ('cdanslair', 0.2589491307735443), ('etes', 0.25741034746170044), ('islamique', 0.2533029019832611)]


In [102]:
result = model_VP.wv.most_similar(positive=['entreprise'], topn=20)
print(result)

[('chefs', 0.5545834898948669), ('economiques', 0.4064718186855316), ('correze', 0.36319443583488464), ('diversite', 0.335783988237381), ('competitivite', 0.3342753052711487), ('entreprises', 0.3171827793121338), ('fonds', 0.30846473574638367), ('producteurs', 0.30189231038093567), ('brive', 0.3006789982318878), ('echanger', 0.2970935106277466), ('intelligence', 0.29270854592323303), ('innovation', 0.28934744000434875), ('modele', 0.28834226727485657), ('liberte', 0.28516390919685364), ('ppe', 0.2845172882080078), ('chaine', 0.28091609477996826), ('choc', 0.2793010473251343), ('mckinsey', 0.27876102924346924), ('bois', 0.2776637077331543), ('production', 0.27419722080230713)]


In [103]:
result = model_VP.wv.most_similar(positive=['economie'], topn=20)
print(result)

[('dettes', 0.4173823893070221), ('faible', 0.38055315613746643), ('enseignement', 0.33700913190841675), ('milliards', 0.33651697635650635), ('migratoires', 0.3338682949542999), ('flux', 0.3253282308578491), ('addition', 0.32450535893440247), ('industrie', 0.32265666127204895), ('ministere', 0.3209927976131439), ('fessenheim', 0.318405419588089), ('securitaire', 0.31498587131500244), ('stopmacron', 0.3146530091762543), ('payer', 0.304373174905777), ('futur', 0.29702746868133545), ('depenses', 0.29565170407295227), ('financement', 0.29510608315467834), ('communautarisme', 0.29309943318367004), ('investissement', 0.2928648293018341), ('innovation', 0.2916550040245056), ('decroissance', 0.291443407535553)]


In [104]:
result = model_VP.wv.most_similar(positive=['emploi'], topn=20)
print(result)

[('formation', 0.5311501622200012), ('insertion', 0.5180622935295105), ('actif', 0.4957907795906067), ('disposition', 0.4420641362667084), ('priorites', 0.4416077733039856), ('professionnelle', 0.4403984248638153), ('revenu', 0.4340096414089203), ('metiers', 0.42482122778892517), ('soins', 0.4089221656322479), ('croissance', 0.40207502245903015), ('relance', 0.4004095196723938), ('regiongrandest', 0.39225488901138306), ('medecine', 0.38313403725624084), ('modernisation', 0.3726564049720764), ('indispensable', 0.3672787547111511), ('projets', 0.3665761649608612), ('attractivite', 0.36569949984550476), ('limite', 0.3586046099662781), ('renforce', 0.35770177841186523), ('pauvrete', 0.3571016490459442)]


In [105]:
result = model_VP.wv.most_similar(positive=['travail'], topn=20)
print(result)

[('travaille', 0.33166971802711487), ('revenus', 0.32865437865257263), ('entier', 0.314595103263855), ('nets', 0.3140977621078491), ('conversion', 0.3067609667778015), ('rtt', 0.3035818040370941), ('salaires', 0.29659849405288696), ('meritent', 0.29090625047683716), ('fruit', 0.2873297929763794), ('augmenteront', 0.28479310870170593), ('salaire', 0.2812255322933197), ('perdre', 0.27821511030197144), ('assistanat', 0.27483099699020386), ('ruraux', 0.2667866051197052), ('dignement', 0.26318690180778503), ('attractivite', 0.25855427980422974), ('redonner', 0.2573591470718384), ('redire', 0.2562735974788666), ('redonnera', 0.2529810070991516), ('diversite', 0.2522284686565399)]


In [106]:
result = model_VP.wv.most_similar(positive=['taxes'], topn=20)
print(result)

[('carburant', 0.6671153903007507), ('carburants', 0.611314594745636), ('essence', 0.5539859533309937), ('litre', 0.5078409314155579), ('tva', 0.46914276480674744), ('baisse', 0.4603436589241028), ('suppression', 0.44208064675331116), ('impot', 0.4269428849220276), ('prix', 0.42059504985809326), ('taxe', 0.3885507881641388), ('baisser', 0.3829054534435272), ('depenser', 0.37887445092201233), ('impots', 0.37587085366249084), ('travaille', 0.3637402057647705), ('deficits', 0.3499810993671417), ('dur', 0.3498794436454773), ('centimes', 0.3482866585254669), ('paye', 0.3476288616657257), ('cout', 0.34315529465675354), ('fruit', 0.3388153314590454)]


In [107]:
result = model_VP.wv.most_similar(positive=['numerique'], topn=20)
print(result)

[('talents', 0.38100969791412354), ('ministere', 0.3743657171726227), ('metiers', 0.3740231394767761), ('favorable', 0.36795273423194885), ('creation', 0.3627491295337677), ('videos', 0.3602665662765503), ('produire', 0.3557143807411194), ('fer', 0.3552454710006714), ('connaitre', 0.34943151473999023), ('innovation', 0.3475528359413147), ('administration', 0.3471928536891937), ('industrielle', 0.31471049785614014), ('supprimerai', 0.31287050247192383), ('gestion', 0.3085949718952179), ('proximite', 0.30501797795295715), ('marion', 0.30377545952796936), ('mizogyne', 0.3013097643852234), ('soignants', 0.299277126789093), ('petites', 0.2983543276786804), ('feminine', 0.2962726354598999)]


In [108]:
result = model_VP.wv.most_similar(positive=['donnees'], topn=20)
print(result)

[('activite', 0.4385002553462982), ('majeure', 0.3631087839603424), ('modernisation', 0.3607379198074341), ('creation', 0.3503059446811676), ('cadre', 0.34962788224220276), ('decision', 0.34787383675575256), ('etablissements', 0.3400760889053345), ('prevention', 0.337658554315567), ('abyssale', 0.32670673727989197), ('aeroport', 0.3246039152145386), ('precedent', 0.3135542571544647), ('medico', 0.3016898036003113), ('direction', 0.301519513130188), ('demander', 0.299934446811676), ('directeur', 0.2993743121623993), ('indigne', 0.29911667108535767), ('formation', 0.29783734679222107), ('pauvrete', 0.2974557876586914), ('appel', 0.2945083975791931), ('meur', 0.29390648007392883)]


In [109]:
result = model_VP.wv.most_similar(positive=['monde'], topn=20)
print(result)

[('allies', 0.3653703033924103), ('gaullienne', 0.2902436852455139), ('voix', 0.267141729593277), ('otan', 0.25944995880126953), ('rassembles', 0.25564152002334595), ('declassement', 0.2548796236515045), ('national', 0.25179266929626465), ('quelle', 0.24928516149520874), ('respectee', 0.2438483089208603), ('president', 0.23864050209522247), ('simplement', 0.23324663937091827), ('americains', 0.22950845956802368), ('forte', 0.22869716584682465), ('vit', 0.22816868126392365), ('efficacite', 0.22732321918010712), ('unis', 0.22121666371822357), ('etats', 0.22098220884799957), ('leve', 0.2204250693321228), ('defense', 0.21953077614307404), ('reve', 0.21321311593055725)]


In [110]:
result = model_VP.wv.most_similar(positive=['realite'], topn=20)
print(result)

[('concret', 0.3169578015804291), ('oui', 0.2994443476200104), ('camp', 0.28076252341270447), ('puisse', 0.27869153022766113), ('aller', 0.2784546911716461), ('connaitre', 0.27111056447029114), ('consideration', 0.26271265745162964), ('bouquet', 0.2551777958869934), ('qofficiel', 0.2527967393398285), ('diplomatie', 0.2523679733276367), ('macronisme', 0.25222375988960266), ('trop', 0.25202009081840515), ('zigzag', 0.24884957075119019), ('croire', 0.24453037977218628), ('hommes', 0.2443080097436905), ('bureau', 0.2429942488670349), ('entendre', 0.2419843226671219), ('facealaguerretf', 0.2413264364004135), ('confiance', 0.24130795896053314), ('islam', 0.2393571436405182)]


In [111]:
result = model_VP.wv.most_similar(positive=['verite'], topn=20)
print(result)

[('ose', 0.39815667271614075), ('nouvelles', 0.3963223695755005), ('aimer', 0.3663409650325775), ('reformes', 0.3229050040245056), ('cheques', 0.3210943341255188), ('servir', 0.31267333030700684), ('vrai', 0.30706942081451416), ('clair', 0.29969966411590576), ('bruno', 0.2915940284729004), ('choix', 0.28767451643943787), ('visage', 0.2860635817050934), ('vouloir', 0.2782474160194397), ('vraies', 0.276400089263916), ('devront', 0.2752382755279541), ('proposer', 0.27325302362442017), ('rassembler', 0.26855188608169556), ('structurelles', 0.268537700176239), ('beaucoup', 0.26557469367980957), ('soutiens', 0.26530230045318604), ('mepris', 0.26420292258262634)]


In [112]:
result = model_VP.wv.most_similar(positive=['societe'], topn=20)
print(result)

[('competence', 0.41619569063186646), ('maintenant', 0.38254109025001526), ('ambitieux', 0.32124653458595276), ('desordre', 0.3010719120502472), ('gerer', 0.28328749537467957), ('experience', 0.2821480333805084), ('faillite', 0.27949291467666626), ('caractere', 0.2782193124294281), ('tient', 0.2681732177734375), ('porte', 0.2635841965675354), ('oublie', 0.2622956931591034), ('difference', 0.2601272165775299), ('presenter', 0.25851118564605713), ('engager', 0.25711843371391296), ('musulmans', 0.2564188838005066), ('civile', 0.2563377320766449), ('concret', 0.25364968180656433), ('anciens', 0.2514747679233551), ('difficultes', 0.2502865195274353), ('ces', 0.24606551229953766)]


In [113]:
result = model_VP.wv.most_similar(positive=['pouvoir'], topn=20)
print(result)

[('net', 0.3256242275238037), ('cesse', 0.32150766253471375), ('espoir', 0.3017944395542145), ('puisse', 0.29651111364364624), ('reprendre', 0.29527023434638977), ('nation', 0.29086869955062866), ('structurelles', 0.28920117020606995), ('votez', 0.27927136421203613), ('reformes', 0.2737451195716858), ('salaire', 0.2658446133136749), ('proposons', 0.2647154629230499), ('mesure', 0.2507406771183014), ('concretes', 0.2502240836620331), ('courage', 0.24860182404518127), ('energie', 0.2383793294429779), ('retablir', 0.23606131970882416), ('vraies', 0.2355472445487976), ('allocations', 0.2345394492149353), ('avance', 0.23331567645072937), ('incarner', 0.23039281368255615)]


In [114]:
result = model_VP.wv.most_similar(positive=['avenir'], topn=20)
print(result)

[('voulez', 0.3539295494556427), ('semaines', 0.3278752565383911), ('soyons', 0.3148824870586395), ('racines', 0.3100893795490265), ('redressement', 0.2747116684913635), ('rentrer', 0.26322266459465027), ('voler', 0.26215842366218567), ('amoureux', 0.2585773468017578), ('comprendre', 0.25755175948143005), ('meilleur', 0.2545199394226074), ('egard', 0.2537539303302765), ('dimanche', 0.2482128143310547), ('culturelle', 0.2457875907421112), ('vital', 0.2432963252067566), ('reelle', 0.24220913648605347), ('presidentiel', 0.24218472838401794), ('dernieres', 0.2403566539287567), ('raisons', 0.24009372293949127), ('medias', 0.23763208091259003), ('cette', 0.2376297116279602)]


In [115]:
result = model_VP.wv.most_similar(positive=['histoire'], topn=20)
print(result)

[('reecrire', 0.3793051540851593), ('dame', 0.34964436292648315), ('vraiment', 0.3438114821910858), ('repentance', 0.3315659761428833), ('faudra', 0.3207544982433319), ('racines', 0.29761865735054016), ('humanite', 0.2949932813644409), ('algerie', 0.2915312945842743), ('culturelle', 0.26999932527542114), ('culture', 0.2677559554576874), ('cede', 0.26209187507629395), ('appeler', 0.2604353427886963), ('reelle', 0.2564195394515991), ('crime', 0.2548016309738159), ('voulez', 0.2519773244857788), ('devenir', 0.2516225278377533), ('reformer', 0.25157785415649414), ('imposer', 0.2492600828409195), ('concitoyens', 0.24899768829345703), ('semaines', 0.24505560100078583)]


In [116]:
result = model_VP.wv.most_similar(positive=['contre'], topn=20)
print(result)

[('lutte', 0.4486585557460785), ('communautarisme', 0.3984121084213257), ('sexisme', 0.3862682580947876), ('sahel', 0.3324112892150879), ('inegalites', 0.32243895530700684), ('illegale', 0.31192705035209656), ('agression', 0.30652913451194763), ('difficiles', 0.30055782198905945), ('fraude', 0.2921205461025238), ('lutter', 0.2878186106681824), ('peine', 0.27320873737335205), ('fakenews', 0.2718193233013153), ('filles', 0.268540620803833), ('combat', 0.2664693593978882), ('continuer', 0.26028409600257874), ('cancer', 0.2593773305416107), ('voie', 0.2572287619113922), ('operation', 0.25641441345214844), ('planchers', 0.2560582756996155), ('cheffe', 0.2556160092353821)]


In [117]:
result = model_VP.wv.most_similar(positive=['faut'], topn=20)
print(result)

[('mon', 0.2748282849788666), ('diriger', 0.2722696363925934), ('reformer', 0.2617682218551636), ('devons', 0.26086872816085815), ('maitrise', 0.26000434160232544), ('semblant', 0.25539493560791016), ('interets', 0.2538970112800598), ('comment', 0.2386554777622223), ('oui', 0.2287578284740448), ('faudra', 0.22646088898181915), ('soigner', 0.2236780971288681), ('etc', 0.22189059853553772), ('appelle', 0.21970883011817932), ('yeux', 0.2147235870361328), ('publiques', 0.21451544761657715), ('tirer', 0.20949074625968933), ('taire', 0.20900461077690125), ('veut', 0.20845608413219452), ('contraire', 0.2070593237876892), ('democratie', 0.20595607161521912)]


In [118]:
result = model_VP.wv.most_similar(positive=['crise'], topn=20)
print(result)

[('sanitaire', 0.43532589077949524), ('actuelle', 0.38411805033683777), ('covid', 0.3548111915588379), ('enjeux', 0.32369694113731384), ('majeur', 0.2991201877593994), ('entendu', 0.29083380103111267), ('confusion', 0.2870453894138336), ('inacceptable', 0.2776019871234894), ('crises', 0.27759793400764465), ('situation', 0.2732791304588318), ('erreur', 0.2673919200897217), ('gewoessner', 0.26728159189224243), ('ukrainienne', 0.25726091861724854), ('financiere', 0.25725096464157104), ('internationale', 0.24926884472370148), ('interieur', 0.24890728294849396), ('dependance', 0.24357280135154724), ('reforme', 0.23997901380062103), ('souverainete', 0.23730258643627167), ('humanitaire', 0.22947277128696442)]
